In [1]:
import pandas as pd
from pathlib import Path

In [2]:
root_path = Path('C:/Users/Hugo Durand/Workspace/data')
CF_file_name = "Template Cash flows TPT_20201231.xlsm"
report_file_name = "20201231_BILMI_BIL Invest Bonds EUR Corporate Investment Grade - I EUR CAP_LU1689730718.xlsx"

In [3]:
cols = "A,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U"
col_names = ["dates", "EUR", "DKK", "HUF", "NOK", "PLN", "RUB", "SEK", "CHF", "GBP", "AUD", "CAD", "HKD", "INR", "JPY", "MYR", "SGD", "KRW", "TWD", "USD"]
print(len(cols.split(',')))
print(len(col_names))
RFR = pd.read_excel(root_path / CF_file_name, sheet_name="rfr", skiprows=9, index_col=0, names=col_names, usecols=cols)
UP = pd.read_excel(root_path / CF_file_name, sheet_name="up", skiprows=9, index_col=0, names=col_names, usecols=cols)
DOWN = pd.read_excel(root_path / CF_file_name, sheet_name="down", skiprows=9, index_col=0, names=col_names, usecols=cols)


20
20


In [4]:
report = pd.read_excel(root_path / report_file_name, sheet_name="Report", skiprows=2)
#print(report.columns.to_list())
report.rename(columns={"Unnamed: 21":"quantity_nominal"}, inplace=True)
report.set_index("14_Identification code of the financial instrument", inplace=True)

In [5]:
isins = report.index.to_list()
isins
instrument = isins[10]
instrument

'FR0013245867'

In [6]:
report.loc[instrument, "12_CIC code of the instrument"]

'FR21'

In [7]:
if report.loc[instrument, "12_CIC code of the instrument"][2] in ["1", "2", "5"]:
    frequency = report.loc[instrument, "38_Coupon payment frequency"]
    coupon = report.loc[instrument, "33_Coupon rate"]
    notional = report.loc[instrument, "quantity_nominal"]
    matdate = report.loc[instrument, "39_Maturity date"]
    cic = report.loc[instrument, "12_CIC code of the instrument"]
    optdate = report.loc[instrument, "43_Call / put date"]
    repdate = report.loc[instrument, "7_Reporting date"]
    strike = report.loc[instrument, "45_Strike price for embedded (call/put) options"]
    currency = report.loc[instrument, "21_Quotation currency (A)"]
    print(frequency)
    print(coupon)
    print(notional)
    print(matdate)
    print(cic)
    print(optdate)
    print(repdate)
    print(strike)
    print(currency)

1.0
1.5
1400000.0
2028-03-27
FR21
2027-12-27
2020-12-31
100.0
EUR


In [8]:
matdates = pd.to_datetime(report["39_Maturity date"], errors='coerce')
matdates.fillna(pd.to_datetime(repdate) + pd.offsets.DateOffset(years=40)) 

14_Identification code of the financial instrument
290018_1       2060-12-31
267100_1       2060-12-31
144120_1       2060-12-31
265000_1       2060-12-31
264293_1       2060-12-31
                  ...    
LU0144746509   2060-12-31
LU1373033965   2060-12-31
FR0012516417   2027-02-09
FR0013346814   2025-07-02
IE00B3T9LM79   2060-12-31
Name: 39_Maturity date, Length: 150, dtype: datetime64[ns]

In [9]:
matdate = matdates.loc[instrument]
repdate = pd.to_datetime(repdate)
optdate = pd.to_datetime(optdate)
matdate = pd.to_datetime(matdate)

In [10]:
offset = pd.DateOffset(months=-12/frequency)

In [11]:
d = matdate + offset
dates = [matdate]
while d > repdate:
    dates = [d] + dates
    d = dates[0] + offset
dates.append(optdate)
dates = pd.DatetimeIndex(dates)
dates = dates[dates <= optdate]
dates[-2]

Timestamp('2027-03-27 00:00:00')

In [31]:
d1 = dates[-2]
d2 = matdate
cash_flow = pd.DataFrame(index=dates, columns=["rfr", "up", "down"])
cash_flow["rfr"] = (coupon / frequency) / 100 * notional
cash_flow["up"] = (coupon / frequency) / 100 * notional
cash_flow["down"] = (coupon / frequency) / 100 * notional
cash_flow.loc[optdate, "rfr"] = (coupon / frequency) / 100 * notional * ((optdate - d1) / (d2 - d1)) + notional * strike / 100
cash_flow.loc[optdate, "up"] = (coupon / frequency) / 100 * notional * ((optdate - d1) / (d2 - d1)) + notional * strike / 100
cash_flow.loc[optdate, "down"] = (coupon / frequency) / 100 * notional * ((optdate - d1) / (d2 - d1)) + notional * strike / 100

cash_flow["rfr"] = cash_flow["rfr"] * RFR.loc[cash_flow.index, currency]
cash_flow["up"] = cash_flow["up"] * UP.loc[cash_flow.index, currency]
cash_flow["down"] = cash_flow["down"] * DOWN.loc[cash_flow.index, currency]
cash_flow

,rfr,up,down
2021-03-27,2.103094e+04,2.098139e+04,2.103094e+04
2022-03-27,2.116289e+04,2.090268e+04,2.116289e+04
2023-03-27,2.129352e+04,2.082220e+04,2.129352e+04
2024-03-27,2.141447e+04,2.073155e+04,2.141447e+04
2025-03-27,2.152285e+04,2.062956e+04,2.152285e+04
2026-03-27,2.161480e+04,2.051171e+04,2.161480e+04
2027-03-27,2.168870e+04,2.037736e+04,2.168870e+04
2027-12-27,1.464855e+06,1.365984e+06,1.464855e+06


In [32]:
RFR.loc[cash_flow.index, currency]

2021-03-27    1.001474
2022-03-27    1.007757
2023-03-27    1.013977
2024-03-27    1.019737
2025-03-27    1.024898
2026-03-27    1.029276
2027-03-27    1.032795
2027-12-27    1.034664
Name: EUR, dtype: float64

In [36]:
DOWN.loc[cash_flow.index, currency]

2021-03-27    1.001474
2022-03-27    1.007757
2023-03-27    1.013977
2024-03-27    1.019737
2025-03-27    1.024898
2026-03-27    1.029276
2027-03-27    1.032795
2027-12-27    1.034664
Name: EUR, dtype: float64

In [33]:
column_97 = 1 - (cash_flow["up"].sum()/cash_flow["rfr"].sum())
column_97 * report.loc[instrument, "26_Valuation weight"]

0.0005659600728310086

In [34]:
column_98 = 1 - (cash_flow["down"].sum()/cash_flow["rfr"].sum())
column_98 * report.loc[instrument, "26_Valuation weight"]

0.0

In [39]:
test = pd.DataFrame(columns=["rfr", "up", "down"])